In [ ]:
function get_ab_phase(co, step, V0, wid, H_Kin, U_0, Degeneracy)
    A = 1
    U_list = []
    for x_t in x0:step:x1
        imps = [ impurity(V0, [x_t,y0], wid) ]
        H1 = Himp(imps, H_Kin)
        E, U = eigenstates(dense(H1))
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    for y_t in y1:step:y2
        imps = [ impurity(V0, [x1,y_t], wid)]
        H1 = Himp(imps, H_Kin)
        E, U = eigenstates(dense(H1))
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    for x_t in x2:-step:x3
        imps = [ impurity(V0, [x_t,y2], wid)]
        H1 = Himp(imps, H_Kin)
        E, U = eigenstates(dense(H1))
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    for y_t in y3:-step:y0
        imps = [ impurity(V0, [x3,y_t], wid)]
        H1 = Himp(imps, H_Kin)
        E, U = eigenstates(dense(H1))
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    AB_Phase = angle(A)/pi
    return AB_Phase, U_list
end

In [ ]:
function get_ab_phase_projection(co, step, V0, wid, H_Kin, U_0, Degeneracy, P1M, P1_dagM, P2M, P2_dagM, basis_tilde, basis_mb_tilde, Int_Operator, basis_tilde2)
    A = 1
    U_list = []
    for x_t in x0:step:x1
        imps = [ impurity(V0, [x_t,y0], wid) ]
        H1 = Himp_proj(imps, H_Kin, P1M, P1_dagM, P2M, P2_dagM, basis_tilde, basis_mb_tilde, Int_Operator, basis_tilde2)
        E, U = eigenstates(dense(H1))
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    for y_t in y1:step:y2
        imps = [ impurity(V0, [x1,y_t], wid)]
        H1 = Himp_proj(imps, H_Kin, P1M, P1_dagM, P2M, P2_dagM, basis_tilde, basis_mb_tilde, Int_Operator, basis_tilde2)
        E, U = eigenstates(dense(H1))
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    for x_t in x2:-step:x3
        imps = [ impurity(V0, [x_t,y2], wid)]
        H1 = Himp_proj(imps, H_Kin, P1M, P1_dagM, P2M, P2_dagM, basis_tilde, basis_mb_tilde, Int_Operator, basis_tilde2)
        E, U = eigenstates(dense(H1))
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    for y_t in y3:-step:y0
        imps = [ impurity(V0, [x3,y_t], wid)]
        H1 = Himp_proj(imps, H_Kin, P1M, P1_dagM, P2M, P2_dagM, basis_tilde, basis_mb_tilde, Int_Operator, basis_tilde2)
        E, U = eigenstates(dense(H1))
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    AB_Phase = angle(A)/pi
    return AB_Phase, U_list
end

In [3]:
function projection_operators(cut_off, basis, U_0, PN)
    substates = 1:cut_off
    
    basis_tilde = SubspaceBasis(basis, U_0[substates])
    basis_tilde2 = basis_tilde⊗basis_tilde
    
    states_mb_tilde = bosonstates(basis_tilde, PN)
    basis_mb_tilde  = ManyBodyBasis(basis_tilde, states_mb_tilde)
    
    P1 = projector(basis, basis_tilde)
    P1_dag = dagger(P1)
    P1M = P1.data
    P1_dagM = P1_dag.data
    
    P2M = (P1⊗P1).data
    P2_dagM = (P1_dag⊗P1_dag).data
    
    return P1M, P1_dagM, P2M, P2_dagM, basis_tilde, basis_tilde2, basis_mb_tilde
end

projection_operators (generic function with 1 method)

In [4]:
function Himp_proj(imps, H_Kin, P1M, P1_dagM, P2M, P2_dagM, basis_tilde, basis_mb_tilde, Int_Operator, basis_tilde2)

    Total_Ham = Himp(imps, H_Kin)

    H1M = Total_Ham.data
    H1_tildeM = P1_dagM * H1M * P1M
    H1_tilde = Operator(basis_tilde, H1_tildeM)
    H1_tilde = (H1_tilde + H1_tilde')/2
    H_NI_tilde_mb = manybodyoperator(basis_mb_tilde, H1_tilde) 
    
    V2M = Int_Operator.data
    V2_tildeM =  P2_dagM * V2M * P2M
    V2_tilde = Operator(basis_tilde2, V2_tildeM)
    Vint_tilde_mb = manybodyoperator(basis_mb_tilde, V2_tilde)

    H_tilde_mb = H_NI_tilde_mb + Vint_tilde_mb
    H_tilde_mb = (H_tilde_mb + H_tilde_mb')/2
    
    return H_tilde_mb
end

Himp_proj (generic function with 1 method)